# Bone Break Classification > 2023-12-07 2:25pm
https://universe.roboflow.com/curso-rphcb/bone-break-classification

Provided by a Roboflow user
License: CC BY 4.0


Source: https://www.kaggle.com/datasets/pkdarabi/bone-break-classification-image-dataset/data